# `HypothesisResearcher`

The role of the `HypothesisResearcher` is to explore leads and hypothesis. More specifically, a `HypothesisResearcher` starts with a research `Lead`, which consists the following:
- a working hypothesis
- a research direction
- observations that led to the hypothesis and research direction

The `HypothesisResearcher` then goes through the following steps to develop (or reject) the hypothesis, and compile reports
1. using the `Lead`, create `info_request`s for the `Librarian`
2. the `Librarian` retrieves relevant document, and the `HypothesisResearcher` processes them in one of two ways:
    - Option 1 `verbatim=False`: the model will answer the `info_request` using the retrieved document, use the `info_retrieved` in subsequent processing
    - Option 2 `verbatim=True`: the model concatenates the retrieved documents as is. Basically like RAG, though I'm a bit afraid of how big the contexts may get.
3. using the `info_requests` and `info_retrieved`, the model compiles a report on the `Lead`.
4. finally, the model decides to do one of the following:
    - if model has reached a certain `iteration`, it stops with a good hypothesis and report
    - if model has not reached the limit but deems the hypothesis is still good enough to pursue, it updates the `Lead` and restarts from step 1
    - if model decides the `Lead` is not worth exploring, stop.

## Initialize

A `HypothesisResearcher` takes a couple different arguments:
- `Librarian`: to retrieve documents
- `research_question`: central research question we are trying to answer
- `new_lead`: the `Lead` that the `HypothesisResearcher` is focusing on
- `iterations`: the number of iterations to explore a `Lead`
- `iterations_used`: this system stores past chat history from developing previous `Lead`s. `iterations_used` tracks how many iterations of the past chat history will be used when developing the current `Lead`. More is probably better but keep in mind too much and context limit explodes.
- `llm` obviously

The other parameters you see in the model are more like properties that the user can access, but i mean if you want to add in your own let's say, `past_leads` or `reports` go for it, not like it'll change anything

In [1]:
from kruppe.llm import OpenAILLM, OpenAIEmbeddingModel

research_question = "What are the key developments and financial projections for Amazon's advertising business, and how is it positioning itself in the digital ad market?"

In [2]:
# the rest are all for the librarian lmao
from kruppe.algorithm.librarian import Librarian
from kruppe.data_source.news.nyt import NewYorkTimesData
from kruppe.functional.rag.index.vectorstore_index import VectorStoreIndex
from kruppe.functional.rag.vectorstore.chroma import ChromaVectorStore
from kruppe.functional.docstore.mongo_store import MongoDBStore

reset_db = False
collection_name="playground_2"

llm = OpenAILLM()
embedding_model = OpenAIEmbeddingModel()

# Create doc store
unique_indices = [['title', 'datasource']] # NOTE: this is important to avoid duplicates
docstore = await MongoDBStore.acreate_db(
    db_name="kruppe_librarian",
    collection_name=collection_name,
    unique_indices=unique_indices,
    reset_db=reset_db
)

# Create vectorstore index
vectorstore = ChromaVectorStore(
    embedding_model=embedding_model,
    collection_name=collection_name,
    persist_path='/Volumes/Lexar/Daniel Liu/vectorstores/kruppe_librarian'
)
if reset_db:
    vectorstore.clear()
    
index = VectorStoreIndex(llm=llm, vectorstore=vectorstore)

# Define news data source
news_source = NewYorkTimesData(headers_path = "/Users/danielliu/Workspace/fin-rag/.nyt-headers.json")

# Define librarian
librarian_llm = OpenAILLM(model="gpt-4o-mini")
librarian = Librarian(
    llm=librarian_llm,
    docstore=docstore,
    index=index,
    news_source=news_source
)

### Get a `Lead`

Usually you'd run `Overseer` to generate leads for a `research_question`, but well i'm too lazy to do that here and I'll just copy one over from `overseer.ipynb`

In [3]:
from kruppe.algorithm.agents import Lead

lead = Lead(
    observation="Amazon's advertising revenue is projected to grow to $55 billion by 2025, yet it currently significantly trails Google and Meta's advertising revenues, estimated at $250 billion and $130 billion respectively in 2023. This wide gap persists despite Amazon's rapid growth and unique data advantages.",
    lead='Investigate the strategies Amazon might employ to scale its advertising revenue more rapidly to close the gap with Google and Meta. Particularly explore how Amazon can leverage its e-commerce platform to create new advertising models that competitors cannot replicate.',
    hypothesis="Amazon will develop innovative advertising formats integrated within its e-commerce transactions that will not only boost engagement but also drive higher conversion rates and revenue, positioning it uniquely against Google and Meta's traditional ad offerings."
)

lead


Lead(observation="Amazon's advertising revenue is projected to grow to $55 billion by 2025, yet it currently significantly trails Google and Meta's advertising revenues, estimated at $250 billion and $130 billion respectively in 2023. This wide gap persists despite Amazon's rapid growth and unique data advantages.", lead='Investigate the strategies Amazon might employ to scale its advertising revenue more rapidly to close the gap with Google and Meta. Particularly explore how Amazon can leverage its e-commerce platform to create new advertising models that competitors cannot replicate.', hypothesis="Amazon will develop innovative advertising formats integrated within its e-commerce transactions that will not only boost engagement but also drive higher conversion rates and revenue, positioning it uniquely against Google and Meta's traditional ad offerings.")

### Initialize `HypothesisResearcher`

In [4]:
from kruppe.algorithm.hypothesis import HypothesisResearcher

hyp_llm = OpenAILLM(model="gpt-4o")
hyp_researcher = HypothesisResearcher(
    llm=hyp_llm,
    research_question=research_question,
    new_lead=lead,
    iterations=3,
    iterations_used=1,
    librarian=librarian
)

## Individual Function Tests

### `create_info_requests`

Very similar as in `BackgroundResearcher`: using the `research_question` and `Lead`, determine the type of information we want to know, and so can submit to the `Librarian`

In [5]:
info_requests = await hyp_researcher.create_info_requests()
info_requests

["1. I want to know the current advertising revenue figures and growth rate for Amazon compared to Google and Meta. Understanding these metrics will help establish a baseline for Amazon's current market position and the extent of the gap it intends to close with its competitors.",
 "2. I want to explore Amazon's existing advertising formats and current integration with its e-commerce platform. This can reveal how Amazon's combination of retail data and customer insights could provide a competitive advantage and the scope for innovative advertisement models.",
 "3. I want to understand consumer behavior and conversion metrics specific to Amazon's advertising within e-commerce settings. Insights into customer interactions with ads during shopping can guide predictions about the effectiveness of new ad formats.",
 "4. I seek insights into Amazon's technological capabilities and infrastructure for supporting advanced and scalable advertising solutions. Knowledge of Amazon’s cloud computing

### `complete_info_request`

Very similar to how `BackgroundResearcher` does it again! Submit a `info_request` to the `Librarian`, retrieve relevant documents, and answer it. 

Note that there is a parameter here called `verbatim`:
- `verbatim=False`: the default mode, where we basically run RAG. Use the retrieved contexts to answer the `info_request` using an llm, and then the response becomes the `info_retrieved`
- `verbatim=True`: we basically concatenate the retrieved contexts as `info_retrieved`, but the size of the retrieved contexts may cause some problem later on when we run `compile_report`

And an additional parameter called `strict`. If `librarian` cannot find anything, `strict=True` will get the response to be a "I do not know", where as `strict=False` will still continue to let LLM generate but without the contexts (essentially relying on vanilla LLM)

In [6]:
# info_request = info_requests[0]
info_request = "1. Investigate Amazon's current advertising revenue streams and growth rates to understand its baseline performance in the digital advertising market. This helps assess how much scaling is needed to approach Google's and Meta's revenue figures."

info_retrieved = await hyp_researcher.complete_info_request(info_request, verbatim = False)

print("Info Request:")
print(info_request)
print("="*10)

print("Info Retrieved:")
print(info_retrieved.text)
print("-"*10)

print("Contexts used:")
for chunk in info_retrieved.sources:
    print(chunk.text, end="\n\n")

Info Request:
1. Investigate Amazon's current advertising revenue streams and growth rates to understand its baseline performance in the digital advertising market. This helps assess how much scaling is needed to approach Google's and Meta's revenue figures.
Info Retrieved:
Amazon's current advertising revenue streams and growth rates can provide critical insights into its baseline performance in the digital advertising market, offering a clearer understanding of how much scaling is necessary to compete with Google and Meta.

Currently, Amazon's advertising model is heavily integrated with its e-commerce platform. The primary revenue streams include:

1. **Sponsored Products and Brands**: These are pay-per-click (PPC) ads that appear alongside search results on Amazon's website. They have proven very successful due to their targeted nature, leveraging Amazon's insights into customer buying behavior.

2. **Display Ads**: These ads are shown to Amazon users as they browse category pages 

### `compile_report_and_update_lead`

This function works by **iteratively improving upon reports and hypothesis previously generated**. As a result, it keeps a record of the messages from past iterations, and appends them to the top of the `messages` list variable passed into llm. However, obviously, there is no past record on the first iteration.

In [7]:
test_info_requests = [info_request]
test_info_retrieved = [info_retrieved]

#### Helper Functions

Note: all of the helper functions use the same `message` variable, and since python is pass by reference when it comes to list, the helper functions will append to the same `message` variable

In [8]:
messages = []

`_compile_report`: Using the research done (which results in two lists, `info_requests` and `info_retrieved`), compile a report built on top of these two and the `Lead`. 

In [ ]:
report = await hyp_researcher._compile_report(
    messages=messages,
    info_requests=test_info_requests,
    info_retrieved=test_info_retrieved
)

print(report.text)

Response(text="**Comprehensive Report on Amazon's Advertising Business Developments**\n\n**Introduction**\n\nThe research question explores the key developments and financial projections for Amazon's advertising business, about its positioning within the digital ad market dominated by Google and Meta. Amazon's advertising revenue, though growing at a remarkable pace, still lags behind these giants. The focus is on investigating Amazon's potential strategic approaches to closing this gap, taking advantage of its unique e-commerce ecosystem to innovate ad formats that stand out from traditional models.\n\n**Evaluation of Current Advertising Revenue Streams**\n\nAmazon's revenue streams in advertising have demonstrated notable growth, contributing to its standing as the third-largest player in digital advertising. The exploration of these revenue streams reveals several key components and opportunities:\n\n1. **Sponsored Products and Brands**: This stream directly leverages Amazon's core 

In [23]:
# LLM's response:
messages[-1]

{'role': 'assistant',
 'content': "**Comprehensive Report on Amazon's Advertising Business Developments**\n\n**Introduction**\n\nThe research question explores the key developments and financial projections for Amazon's advertising business, about its positioning within the digital ad market dominated by Google and Meta. Amazon's advertising revenue, though growing at a remarkable pace, still lags behind these giants. The focus is on investigating Amazon's potential strategic approaches to closing this gap, taking advantage of its unique e-commerce ecosystem to innovate ad formats that stand out from traditional models.\n\n**Evaluation of Current Advertising Revenue Streams**\n\nAmazon's revenue streams in advertising have demonstrated notable growth, contributing to its standing as the third-largest player in digital advertising. The exploration of these revenue streams reveals several key components and opportunities:\n\n1. **Sponsored Products and Brands**: This stream directly leve

`_evaluate_lead`: Then, evaluate the `lead` and see if it is still worth exploring. If it is not, return `False`.

In [24]:
continue_lead = await hyp_researcher._evaluate_lead(messages)
continue_lead

True

In [25]:
# LLM's response:
messages[-1]

{'role': 'assistant',
 'content': "Accept. The newly retrieved information reinforces the potential for Amazon to develop innovative advertising formats that leverage its e-commerce transactions. The data shows that Amazon's current revenue streams, such as Sponsored Products and Brands, already capitalizes on the purchasing mindset of consumers, providing a framework for enhanced engagement and conversion. Additionally, Amazon's DSP and the growing trend in video and streaming ads on platforms like Prime Video and Twitch offer further avenues for innovation. While the analysis points to areas where expansion and integration across various Amazon services could bolster this strategy, the foundational hypothesis remains valid and is supported by Amazon's strategic advantages in data utilization and conversion-focused advertising. These factors align with the hypothesis, suggesting Amazon is well-positioned to differentiate itself from competitors like Google and Meta, which focus on mor

`_update_lead`: If it is worth exploring, we want to update the `lead` to get brand new hypothesis and research direction. Note that if the `iteration` has reached the end, then we also stop (but only after updating the hypothesis).


In [26]:
new_lead = await hyp_researcher._update_lead(messages)
new_lead

Lead(observation="Amazon's advertising revenue continues to grow through formats closely integrated with consumer purchasing decisions, such as Sponsored Products and Brands. This integration capitalizes on the direct path from engagement to transaction, suggesting an emerging capability for Amazon to leverage its unique data advantage in e-commerce to create highly conversion-focused advertising experiences.", lead='Amazon is poised to expand its dominance in the digital ad market by creating advertisement models that use its transactional data to offer unparalleled consumer insight and engagement, integrating these models seamlessly across its platform, including services like Prime Video, Twitch, and Alexa, to create a fully cohesive consumer advertising journey that competitors cannot easily replicate.', hypothesis='Investigate how Amazon can further integrate its advertising capabilities across its entire ecosystem, particularly examining opportunities in emerging technologies and

In [9]:
# LLM's response:
messages[-1]

IndexError: list index out of range

#### Main Function Call

In [10]:
continue_research = await hyp_researcher.compile_report_and_update_lead(
    info_requests=test_info_requests,
    info_retrieved=test_info_retrieved,
)

continue_research

True

In [12]:
# some info that changed:
print("Iterations remaining:", hyp_researcher.iterations)
print("Message history length:", len(hyp_researcher._messages_history_list))
print("Latest hypothesis:", hyp_researcher.latest_hypothesis)
print("Latest report:", hyp_researcher.latest_report)

Iterations remaining: 2
Message history length: 1
Latest hypothesis: Investigate AI’s role in personalizing Amazon's advertising strategies to understand how these technologies can enhance consumer interaction and conversion at the point of purchase within their platform. Additionally, explore which specific interactive ad formats and innovations are most effective in driving engagement and sales directly on Amazon.
Latest report: **Comprehensive Report: Amazon's Advertising Business Growth and Strategic Positioning**

**1. Introduction**
The digital advertising market is dominated by Google and Meta, leaving significant room for Amazon, whose advertising revenue is projected to reach $55 billion by 2025, to catch up. With its unique position as an e-commerce giant, Amazon has the potential to innovate new ad formats, leveraging its data-centric approach to disrupt traditional advertising.

**2. Baseline Performance Analysis**
Amazon's advertising performance is primarily rooted in its

## `execute`

aight let's try the real thing...

In [13]:
from kruppe.algorithm.hypothesis import HypothesisResearcher

hyp_llm = OpenAILLM(model="gpt-4o")
hyp_researcher = HypothesisResearcher(
    llm=hyp_llm,
    research_question=research_question,
    new_lead=lead,
    iterations=3,
    iterations_used=1,
    librarian=librarian
)

In [14]:
latest_report = await hyp_researcher.execute() # note: i can also get this from hyp_researcher.latest_report
latest_report

Investigate lead:   0%|          | 0/3 [00:00<?, ?it/s]Retrieving from library for info request: 1. I want to know about the current and emerging advertising formats that Amazon is employing within its e-commerce platform. Understanding specific ad types, such as Sponsored Products, Display Ads, and Video Formats, will reveal how they differ from traditional digital ads and illustrate potential innovations that can uniquely position Amazon against competitors.
ERROR:kruppe.algorithm.librarian:Could not determine relevance score from LLM. LLM response: [thought process]
The user's inquiry focuses on understanding the legal and regulatory environment surrounding digital advertising, specifically regarding Amazon and its competition with Google and Meta. The provided contexts contain various discussions about lawsuits and regulatory actions taken against tech giants including Google, Apple, Meta, and Amazon itself. There are references to antitrust lawsuits and the impact of legal decisio

Response(text="### Comprehensive Report: Evaluating Amazon's AI-Driven Advertising Strategy\n\n#### Introduction\nThis report synthesizes findings regarding Amazon's use of AI and machine learning technologies to enhance its advertising business. We explore how these technologies are implemented across different global markets, assess Amazon's competitive positioning, and consider the impact of regulatory environments and consumer perceptions.\n\n#### Synthesis of New Information\n\n**AI Technologies Deployment:**\n- Amazon employs AI-driven tools such as Sponsored Product Listings, Personalized Video and Display Ads, and its DSP (Demand-Side Platform) to target consumer segments globally. These technologies leverage customer data for localized ad experiences but specific tool details remain sparse.\n\n**Localization Strategies:**\n- Amazon adapts its advertising approach in international markets, notably in India and Japan. By using locally relevant languages and aligning ads with reg

In [15]:
hyp_researcher._lead_status

1

In [17]:
print(hyp_researcher.latest_hypothesis)

Investigate the relationship between Amazon's localized AI-driven advertising strategies and changes in consumer trust and brand perception. Examine how these factors influence long-term customer loyalty and Amazon's ability to sustain its market share expansion against competitors like Google and Meta.


In [19]:
print(hyp_researcher.latest_report.text)

### Comprehensive Report: Evaluating Amazon's AI-Driven Advertising Strategy

#### Introduction
This report synthesizes findings regarding Amazon's use of AI and machine learning technologies to enhance its advertising business. We explore how these technologies are implemented across different global markets, assess Amazon's competitive positioning, and consider the impact of regulatory environments and consumer perceptions.

#### Synthesis of New Information

**AI Technologies Deployment:**
- Amazon employs AI-driven tools such as Sponsored Product Listings, Personalized Video and Display Ads, and its DSP (Demand-Side Platform) to target consumer segments globally. These technologies leverage customer data for localized ad experiences but specific tool details remain sparse.

**Localization Strategies:**
- Amazon adapts its advertising approach in international markets, notably in India and Japan. By using locally relevant languages and aligning ads with regional cultural events, Ama